In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
import plotly.graph_objects as go

# Sidebar for user input
st.sidebar.title("Window Scheduling Dashboard")
uploaded_file = st.sidebar.file_uploader("Upload GenerationData.xlsx", type="xlsx")

if uploaded_file:
    # Step 1: Load and filter data
    df = pd.read_excel(uploaded_file)
    df.columns = ['Date', 'Type'] + [f'Time_{i}' for i in range(1, len(df.columns) - 1)]
    df = df[df['Type'] == 'Scheduled Generation (MW)'].drop(columns=['Type'])
    st.success("Step 1: Data Loaded and Filtered")
    st.dataframe(df.head())

    # Step 2: Reshape to time series
    df_melted = df.melt(id_vars=['Date'], var_name='TimeBlock', value_name='Generation')
    time_blocks = pd.date_range(start="00:00", periods=96, freq="15min").time
    time_block_mapping = dict(zip([f'Time_{i}' for i in range(1, 97)], time_blocks))
    df_melted['Time'] = df_melted['TimeBlock'].map(time_block_mapping)
    df_melted['Datetime'] = pd.to_datetime(df_melted['Date'].astype(str)) + pd.to_timedelta(df_melted['Time'].astype(str))
    df_timeseries = df_melted[['Datetime', 'Generation']].sort_values('Datetime').reset_index(drop=True)

    st.success("Step 2: Reshaped to Time Series")
    st.dataframe(df_timeseries.head())

    # Step 3: Resample to daily
    daily_df = df_timeseries.resample('D', on='Datetime').mean().reset_index()
    st.success("Step 3: Resampled to Daily Averages")
    st.line_chart(daily_df.set_index('Datetime'))

    # Step 4: STL Decomposition
    st.subheader("STL Decomposition")
    st.write("Extracting trend and seasonality")
    result = STL(daily_df.set_index('Datetime')['Generation'], period=365).fit()

    # STL Plot
    fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
    axs[0].plot(result.trend)
    axs[0].set_title("Trend")
    axs[1].plot(result.seasonal)
    axs[1].set_title("Seasonality")
    axs[2].plot(result.resid)
    axs[2].set_title("Residual")
    st.pyplot(fig)

    # Optional Plotly chart
    st.subheader("Interactive Plotly View")
    fig_plotly = go.Figure()
    fig_plotly.add_trace(go.Scatter(x=daily_df['Datetime'], y=daily_df['Generation'], mode='lines', name='Daily Avg Gen'))
    fig_plotly.update_layout(title="Daily Scheduled Generation", xaxis_title="Date", yaxis_title="MW")
    st.plotly_chart(fig_plotly, use_container_width=True)
else:
    st.warning("👈 Upload your Excel file from sidebar to begin.")


2025-06-18 12:08:31.242 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:31.715 
  command:

    streamlit run C:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-06-18 12:08:31.715 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:31.716 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:31.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:31.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:31.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-18 12:08:3